In [2]:
import ROOT
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import numpy as np
from os.path import exists
# Get all target polarizations from RCDB
import sys
sys.path.append("/work/clas12/users/gmat/packages/clas12root/rcdb/python/")
import rcdb
from rcdb.provider import RCDBProvider
from rcdb.model import ConditionType
from scipy.optimize import curve_fit
db = RCDBProvider("mysql://rcdb@clasdb/rcdb")

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-btr36g2x because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to JupyROOT 6.22/06


In [ ]:
def get_csv(filepath, run):
    return filepath + "run{}_RUNScaler.csv".format(run) , filepath + "run{}_HELScaler.csv".format(run)
def get_csv_all(filepath, run):
    return filepath + "run{}_RUNScaler-all.csv".format(run) , filepath + "run{}_HELScaler-all.csv".format(run)

In [ ]:
def get_from_rcdb(run,param):
    r = db.get_run(run)
    cnd = r.get_condition(param)
    return cnd.value

In [ ]:
def print_problem(bank,df):
    nfiles = 1+np.amax(df["fileidx"].unique())
    if(bank=="RUN"):
        print("-------------------------------------------")
        print("RUN NUMBER: ".ljust(12),df["run"][0])
        print("SCALER: ".ljust(12),"RUN")
        print("NFILES: ".ljust(12),nfiles)
        mean = np.mean(df["fcg_diff"].to_numpy())
        std = np.std(df["fcg_diff"].to_numpy())
        print("MEAN FCUPGATED DIFF: ".ljust(24),mean)
        print("STDDEV FCUPGATED DIFF: ".ljust(24),std)
        df["fcg_u"]=np.abs(df["fcg_diff"]-mean)
        outliers2std = df[df["fcg_u"]>2*std]
        outliers3std = df[df["fcg_u"]>3*std]
        outliers2not3std = df[(df["fcg_u"]>2*std) & (df["fcg_u"]<3*std)]
        print("OUTLIERS (2 STDDEV): ".ljust(24),outliers2std["fileidx"].to_list())
        print("        N STDDEV'S : ".ljust(24),np.abs(np.round((outliers2std["fcg_diff"].to_numpy()-mean)/std,2)))
        print("OUTLIERS (3 STDDEV): ".ljust(24),outliers3std["fileidx"].to_list())
        print("        N STDDEV'S : ".ljust(24),np.abs(np.round((outliers3std["fcg_diff"].to_numpy()-mean)/std,2)))
        print("-------------------------------------------")
        return outliers2not3std["fileidx"].to_list(),outliers2not3std["fcg_diff"].to_list(), outliers3std["fileidx"].to_list(),outliers3std["fcg_diff"].to_list(),mean,std
    
    if(bank=="HEL"):
            print("-------------------------------------------")
            print("RUN NUMBER: ".ljust(12),df["run"][0])
            print("SCALER: ".ljust(12),"HEL")
            print("NFILES: ".ljust(12),nfiles)
            df["fcg_diff"]=(df['fcupgated_total_pos']-df['fcupgated_total_neg'])/(df['fcupgated_total_pos']+df['fcupgated_total_neg'])
            mean = np.mean(df["fcg_diff"].to_numpy())
            std = np.std(df["fcg_diff"].to_numpy())
            print("MEAN FCUPGATED DIFF: ".ljust(24),mean)
            print("STDDEV FCUPGATED DIFF: ".ljust(24),std)
            df["fcg_u"]=np.abs(df["fcg_diff"]-mean)
            outliers2std = df[df["fcg_u"]>2*std]
            outliers3std = df[df["fcg_u"]>3*std]
            outliers2not3std = df[(df["fcg_u"]>2*std) & (df["fcg_u"]<3*std)]
            print("OUTLIERS (2 STDDEV): ".ljust(24),outliers2std["fileidx"].to_list())
            print("        N STDDEV'S : ".ljust(24),np.abs(np.round((outliers2std["fcg_diff"].to_numpy()-mean)/std,2)))
            print("OUTLIERS (3 STDDEV): ".ljust(24),outliers3std["fileidx"].to_list())
            print("        N STDDEV'S : ".ljust(24),np.abs(np.round((outliers3std["fcg_diff"].to_numpy()-mean)/std,2)))
            print("-------------------------------------------")
            return outliers2not3std["fileidx"].to_list(),outliers2not3std["fcg_diff"].to_list(), outliers3std["fileidx"].to_list(),outliers3std["fcg_diff"].to_list(),mean,std

# MAIN CODE
---

In [2]:
plt.rcParams['text.usetex'] = False

def scaler_ana_singleRun(filepath = "",
                         run = 0,
                         correction = False):
    # Default filepath
    if(filepath==""):
        filepath = "/volatile/clas12/users/gmat/clas12analysis.sidis.data/rgc-scaler/"
        
    # Get CSV scalers
    csvRUN, csvHEL = get_csv(filepath,run)
    csvRUNall, csvHELall = get_csv_all(filepath,run)
    

    # Create dataframes
    dfRUN = pd.read_csv(csvRUN,header=None,
                        names=["run","fileidx","entries","fcupgated_min","fcupgated_max",
                              "fcg_diff","livetime_total","livetime_avg","fcup_min","fcup_max","fc_diff","livetime_avg*fc_diff"])
    
    dfHEL = pd.read_csv(csvHEL,header=None,
                        names=["run","fileidx","entries","fcupgated_total","fcupgated_total_pos",
                              "fcupgated_total_neg","fcupgated_total_zero","raw_fcupgated_total",
                              "raw_fcupgated_total_pos","raw_fcupgated_total_neg","raw_fcupgated_total_zero"])

    
    if(correction):
        dfHEL["fcupgated_total"]=dfHEL["raw_fcupgated_total"]
        dfHEL["fcupgated_total_pos"]=dfHEL["raw_fcupgated_total_pos"]
        dfHEL["fcupgated_total_neg"]=dfHEL["raw_fcupgated_total_neg"]
        dfHEL["fcupgated_total_zero"]=dfHEL["raw_fcupgated_total_zero"]
    
    # Grab RCDB values of interest
    Tpol = get_from_rcdb(run,'target_polarization')
    starttime = get_from_rcdb(run,'run_start_time')
    stoptime = get_from_rcdb(run,'run_end_time')
    runtime = (stoptime-starttime).seconds
    HWP = get_from_rcdb(run,'half_wave_plate')
    if(HWP==0): # If the HWP is in, swap HEL::scaler columns
        dfHEL["temp"]=dfHEL["fcupgated_total_pos"]
        dfHEL["fcupgated_total_pos"]=dfHEL["fcupgated_total_neg"]
        dfHEL["fcupgated_total_neg"]=dfHEL["temp"]
        
        
    # Find problematic fileidx's
    # ol represents "outliers"
    ol2_RUN_x,ol2_RUN_y,ol3_RUN_x,ol3_RUN_y,RUN_mean,RUN_std = print_problem("RUN",dfRUN)
    ol2_HEL_x,ol2_HEL_y,ol3_HEL_x,ol3_HEL_y,HEL_mean,HEL_std = print_problem("HEL",dfHEL)
    
    # ----------------------------
    # RUN::scaler Plots
    # ----------------------------
    # Entries vs. fileidx
    fig0, axs0 = plt.subplots(1,1,figsize=(5,5))
    axs0.plot(dfRUN["fileidx"],dfRUN["entries"],"k-")
    axs0.set_xlabel("File Number")
    axs0.set_ylabel("Scaler bank entries")
    axs0.grid()
    
    # Fcup plots vs. fileidx
    fig1, axs1 = plt.subplots(2,2,figsize=(15, 10))
    axs1[0,0].plot(dfRUN["fileidx"],dfRUN["fcupgated_min"],"k-",label="FCgated_min")
    axs1[0,0].plot(dfRUN["fileidx"],dfRUN["fcupgated_max"],"r--",label="FCgated_max")
    axs1[0,0].set_xlabel("File Number")
    axs1[0,0].set_ylabel("Faraday Cup Accumulation (Gated)")
    axs1[0,0].grid()
    axs1[0,0].legend()
    
    axs1[0,1].plot(dfRUN["fileidx"],dfRUN["fcg_diff"],"k-")
    axs1[0,1].set_xlabel("File Number")
    axs1[0,1].set_ylabel("FC gated (max-min)")
    axs1[0,1].grid()
    
    axs1[1,0].plot(dfRUN["fileidx"],dfRUN["fcup_min"],"k-",label="FC_min")
    axs1[1,0].plot(dfRUN["fileidx"],dfRUN["fcup_max"],"r--",label="FC_max")
    axs1[1,0].set_xlabel("File Number")
    axs1[1,0].set_ylabel("Faraday Cup Accumulation (Not Gated)")
    axs1[1,0].grid()
    axs1[1,0].legend()
    
    axs1[1,1].plot(dfRUN["fileidx"],dfRUN["fc_diff"],"k-")
    axs1[1,1].set_xlabel("File Number")
    axs1[1,1].set_ylabel("FC not gated (max-min)")
    axs1[1,1].grid()
    
    for r in range(2):
        for c in range(2):
            axs1[r,c].annotate('Run {}'.format(run), xy=(0.05, 0.8), xycoords='axes fraction',color="b")
            axs1[r,c].annotate('Reading from RUN::scaler', xy=(0.05, 0.75), xycoords='axes fraction',color="0.3")
            axs1[r,c].annotate('Target Pol = {:.3f}'.format(Tpol),xy=(0.05, 0.7), xycoords='axes fraction',color="0.3")
            axs1[r,c].annotate('Runtime = {:.1f}min'.format(runtime/60),xy=(0.05, 0.65), xycoords='axes fraction',color="0.3")

    fig1.savefig("./plots/scaler/p1_{}.pdf".format(run))
    
    # Everything vs. Fileidx
    fig2, axs2 = plt.subplots(3,3,figsize=(15, 15))
    cols2 = dfRUN.columns.to_list()[3:]
    for r in range(3):
        for c in range(3):
            dfRUN.plot(x="fileidx",y=cols2[3*r+c],ax=axs2[r,c],style="k-")
            axs2[r,c].set_xlabel('File Number')
            axs2[r,c].annotate('Run {}'.format(run), xy=(0.05, 0.85), xycoords='axes fraction',color="b")
            axs2[r,c].annotate('Reading from RUN::scaler', xy=(0.05, 0.8), xycoords='axes fraction',color="0.3")
            axs2[r,c].annotate('Target Pol = {:.3f}'.format(Tpol),xy=(0.05, 0.75), xycoords='axes fraction',color="0.3")
            axs2[r,c].annotate('Runtime = {:.1f}min'.format(runtime/60),xy=(0.05, 0.7), xycoords='axes fraction',color="0.3")
            axs2[r,c].grid()
            
    fig2.savefig("./plots/scaler/p2_{}.pdf".format(run))
    # Comparing FCupgated difference and FCup weighted by livetime
    fig3,axs3 = plt.subplots(1,1,figsize=(5,5))
    axs3.plot(dfRUN['fileidx'],dfRUN['fcg_diff']-dfRUN['livetime_avg*fc_diff'],"k-")
    #axs3.set_ylim(-1,1)
    axs3.set_xlabel("File Number")
    axs3.set_ylabel("")
    axs3.set_title("(FCup Gated Diff) - (FCup Diff * AvgLivetime)")
    axs3.annotate('Run {}'.format(run), xy=(0.05, 0.95), xycoords='axes fraction',color="b")
    axs3.annotate('Reading from RUN::scaler', xy=(0.05, 0.91), xycoords='axes fraction',color="0.3")
    axs3.annotate('Target Pol = {:.3f}'.format(Tpol),xy=(0.05, 0.87), xycoords='axes fraction',color="0.3")
    axs3.annotate('Runtime = {:.1f}min'.format(runtime/60),xy=(0.05, 0.83), xycoords='axes fraction',color="0.3")
    axs3.axhline(y=0, color='0.7', linestyle='--',zorder=10)
    fig3.savefig("./plots/scaler/p3_{}.pdf".format(run))
    
    # Showcase outliers in RUN::scaler
    fig4, axs4 = plt.subplots(1,1,figsize=(5, 5))
    axs4.plot(dfRUN["fileidx"],dfRUN["fcg_diff"],"k-",label="FCgated_diff")
    axs4.plot(ol2_RUN_x,ol2_RUN_y,"go",markersize=20,fillstyle='none',label=r"{} Outliers > 2$\sigma$".format(len(ol2_RUN_x)))
    axs4.plot(ol3_RUN_x,ol3_RUN_y,"ro",markersize=20,fillstyle='none',label=r"{} Outliers > 3$\sigma$".format(len(ol3_RUN_x)))
    axs4.axhline(y=RUN_mean, color='0.7', linestyle='--',zorder=10)
    axs4.axhspan(RUN_mean-2*RUN_std, RUN_mean+2*RUN_std, color="green", alpha=0.1,label=r"2$\sigma$ region")
    axs4.axhspan(RUN_mean+2*RUN_std, RUN_mean+3*RUN_std, color="red", alpha=0.1,label=r"3$\sigma$ region")
    axs4.axhspan(RUN_mean-3*RUN_std, RUN_mean-2*RUN_std, color="red", alpha=0.1)
    axs4.set_xlabel("File Number")
    axs4.set_ylabel("Faraday Cup Accumulation Difference (Gated)")
    axs4.grid()
    axs4.legend(markerscale=0.5)
    fig4.savefig("./plots/scaler/p4_{}.pdf".format(run))
    
    
    
    # ----------------------------
    # HEL::scaler Plots
    # ----------------------------
    
    # Everything vs. Fileidx
    fig10, axs10 = plt.subplots(2,2,figsize=(15, 15))
    cols10 = dfHEL.columns.to_list()[3:]
    for r in range(2):
        for c in range(2):
            dfHEL.plot(x="fileidx",y=cols10[2*r+c],ax=axs10[r,c],style="k-")
            axs10[r,c].set_xlabel('File Number')
            axs10[r,c].annotate('Run {}'.format(run), xy=(0.65, 0.85), xycoords='axes fraction',color="b")
            axs10[r,c].annotate('Reading from HEL::scaler', xy=(0.65, 0.8), xycoords='axes fraction',color="0.3")
            axs10[r,c].annotate('Target Pol = {:.3f}'.format(Tpol),xy=(0.65, 0.75), xycoords='axes fraction',color="0.3")
            axs10[r,c].annotate('Runtime = {:.1f}min'.format(runtime/60),xy=(0.65, 0.7), xycoords='axes fraction',color="0.3")
            axs10[r,c].grid()
    fig10.savefig("./plots/scaler/p10_{}.pdf".format(run))
    # Comparing FCupgated difference and FCup weighted by livetime
    fig11,axs11 = plt.subplots(1,2,figsize=(15,7.5))
    dfHEL['fcg_diff']=dfHEL['fcupgated_total_pos']-dfHEL['fcupgated_total_neg']
    dfHEL['fcg_tot']=dfHEL['fcupgated_total_pos']+dfHEL['fcupgated_total_neg']
    axs11[0].plot(dfHEL['fileidx'],dfHEL['fcg_diff'],"k-",label="Norm. FCup Asym.")
    axs11[0].set_xlabel("File Number")
    axs11[0].set_ylabel(r"FC$^{+}$ - FC$^{-}$")
    axs11[0].set_title("Faraday Cup Gated Asymmetry")
    axs11[0].annotate('Run {}'.format(run), xy=(0.05, 0.95), xycoords='axes fraction',color="b")
    axs11[0].annotate('Reading from HEL::scaler', xy=(0.05, 0.91), xycoords='axes fraction',color="0.3")
    axs11[0].annotate('Target Pol = {:.3f}'.format(Tpol),xy=(0.05, 0.87), xycoords='axes fraction',color="0.3")
    axs11[0].annotate('Runtime = {:.1f}min'.format(runtime/60),xy=(0.05, 0.83), xycoords='axes fraction',color="0.3")
    axs11[0].axhline(y=0, color='0.7', linestyle='--',zorder=10)

    axs11[1].plot(dfHEL['fileidx'],dfHEL['fcg_diff']/dfHEL['fcg_tot'],"k-")
    axs11[1].plot(ol2_HEL_x,ol2_HEL_y,"go",markersize=20,fillstyle='none',label=r"{} Outliers > 2$\sigma$".format(len(ol2_HEL_x)))
    axs11[1].plot(ol3_HEL_x,ol3_HEL_y,"ro",markersize=20,fillstyle='none',label=r"{} Outliers > 3$\sigma$".format(len(ol3_HEL_x)))
    axs11[1].set_xlabel("File Number")
    axs11[1].set_ylabel(r"$\frac{FC^{+}-FC^{-}}{FC^{+}+FC^{-}}$",fontsize=20)
    axs11[1].set_title("Normalized Faraday Cup Gated Asymmetry")
    axs11[1].annotate('Run {}'.format(run), xy=(0.05, 0.95), xycoords='axes fraction',color="b")
    axs11[1].annotate('Reading from HEL::scaler', xy=(0.05, 0.92), xycoords='axes fraction',color="0.3")
    axs11[1].annotate('Target Pol = {:.3f}'.format(Tpol),xy=(0.05, 0.89), xycoords='axes fraction',color="0.3")
    axs11[1].annotate('Runtime = {:.1f}min'.format(runtime/60),xy=(0.05, 0.86), xycoords='axes fraction',color="0.3")
    axs11[1].axhspan(HEL_mean-2*HEL_std, HEL_mean+2*HEL_std, color="green", alpha=0.1,label=r"2$\sigma$ region")
    axs11[1].axhspan(HEL_mean+2*HEL_std, HEL_mean+3*HEL_std, color="red", alpha=0.1,label=r"3$\sigma$ region")
    axs11[1].axhspan(HEL_mean-3*HEL_std, HEL_mean-2*HEL_std, color="red", alpha=0.1)
    axs11[1].axhline(y=0, color='0.7', linestyle='--',zorder=10)
    axs11[1].legend(markerscale=0.5)
    fig11.savefig("./plots/scaler/p11_{}.pdf".format(run))
    
    # FCup_gated average evolution
    # Include skips?
    doSkips=False
    mini=0
    if(doSkips):
        mini=np.amin(dfHEL[dfHEL["fcupgated_total_pos"]<1]["fileidx"].to_numpy())
    
    fcg_arr = dfHEL['fcg_diff'].to_numpy()[mini:]/dfHEL['fcg_tot'].to_numpy()[mini:]
    fcg_array_n = dfHEL['fcg_diff'].to_numpy()[mini:]
    fcg_array_d = dfHEL['fcg_tot'].to_numpy()[mini:]
    fcg_bool_2std = np.abs(fcg_arr-HEL_mean)<2*HEL_std 
    fcg_array_2std_n = fcg_array_n[fcg_bool_2std]
    fcg_array_2std_d = fcg_array_d[fcg_bool_2std]
    fcg_bool_3std = np.abs(fcg_arr-HEL_mean)<3*HEL_std
    fcg_array_3std_n = fcg_array_n[fcg_bool_3std]
    fcg_array_3std_d = fcg_array_d[fcg_bool_3std]
    fcg_ = []
    fcg_2std = []
    fcg_3std = []
    
    
    for i in range(len(fcg_arr)):
        fcg_.append(np.sum(fcg_array_n[:i+1])/np.sum(fcg_array_d[:i+1]))
    j=0
    for i in range(len(fcg_bool_2std)):
        if(fcg_bool_2std[i]==False):
            if(fcg_2std==[]):
                fcg_2std.append(0)
            else:
                fcg_2std.append(fcg_2std[i-1])
        else:
            fcg_2std.append(np.sum(fcg_array_2std_n[:i+1])/np.sum(fcg_array_2std_d[:i+1]))
            j+=1
    j=0
    for i in range(len(fcg_bool_3std)):
        if(fcg_bool_3std[i]==False):
            if(fcg_3std==[]):
                fcg_3std.append(0)
            else:
                fcg_3std.append(fcg_3std[i-1])
        else:
            fcg_3std.append(np.sum(fcg_array_3std_n[:i+1])/np.sum(fcg_array_3std_d[:i+1]))
            j+=1
    
    fig12,axs12 = plt.subplots(1,1,figsize=(10,10))
    axs12.plot(range(len(fcg_arr)),fcg_,"k-",label="All")
    axs12.plot(range(len(fcg_arr)),fcg_2std,"g-",label=r"Removed 2$\sigma$ Outliers")
    axs12.plot(range(len(fcg_arr)),fcg_3std,"r-",label=r"Removed 3$\sigma$ Outliers")
    axs12.set_xlabel("File Number")
    axs12.set_ylabel("Cumulative FC Asymmetry",fontsize=20)
    axs12.set_title("Evolution of the FC Asymmetry")
    axs12.annotate('Run {}'.format(run), xy=(0.5, 0.95), xycoords='axes fraction',color="b")
    axs12.annotate('Reading from HEL::scaler', xy=(0.5, 0.92), xycoords='axes fraction',color="0.3")
    axs12.annotate('Target Pol = {:.3f}'.format(Tpol),xy=(0.5, 0.89), xycoords='axes fraction',color="0.3")
    axs12.annotate('Runtime = {:.1f}min'.format(runtime/60),xy=(0.5, 0.86), xycoords='axes fraction',color="0.3")
    axs12.annotate('Number Outliers = {}'.format(len(ol2_HEL_x)),xy=(0.5, 0.83), xycoords='axes fraction',color="0.3")
    axs12.annotate('(ALL) Final Asym = {:.5f}'.format(fcg_[-1]),xy=(0.5, 0.65), xycoords='axes fraction',color="k")
    axs12.annotate(r'(REMOVED 2$\sigma$) Final Asym = {:.5f}'.format(fcg_2std[-1]),xy=(0.5, 0.62), xycoords='axes fraction',color="g")
    axs12.annotate(r'(REMOVED 3$\sigma$) Final Asym = {:.5f}'.format(fcg_3std[-1]),xy=(0.5, 0.59), xycoords='axes fraction',color="r")
    axs12.legend()
    axs12.grid()
    fig12.savefig("./plots/scaler/p12_{}.pdf".format(run))
    

NameError: name 'plt' is not defined

In [ ]:
def unpack_asym(filepath = "",
                   runMin = 0,
                   runMax = 0):
    
    df = pd.DataFrame(columns=["run","HWP","Tpol","clockgated_asym","fcupgated_asym","clockgated_asym_corr","fcupgated_asym_corr"])
    r = []
    HWP = []
    Tpol = []
    # Without correction
    clockgated_asym_numerator = []
    clockgated_asym_denominator = []
    clockgated_asym = []
    clockgated_pos = []
    clockgated_neg = []
    fcupgated_asym_numerator = []
    fcupgated_asym_denominator = []
    fcupgated_asym = []
    fcupgated_pos = []
    fcupgated_neg = []
    
    # With correction
    clockgated_asym_numerator_corr = []
    clockgated_asym_denominator_corr = []
    clockgated_asym_corr = []
    clockgated_pos_corr = []
    clockgated_neg_corr = []
    fcupgated_asym_numerator_corr = []
    fcupgated_asym_denominator_corr = []
    fcupgated_asym_corr = []
    fcupgated_pos_corr = []
    fcupgated_neg_corr = []
    
    hel = 0
    zhat = 1
    for run in np.arange(runMin,runMax+1):
        # Get CSV scalers
        csvRUNall, csvHELall = get_csv_all(filepath,run)
        if(not exists(csvRUNall)):
            continue
        dfHELall = pd.read_csv(csvHELall)
        display(dfHELall)
        return -1
        if(dfHELall.loc[dfHELall.helicity==zhat,"fcupgated"].sum()==0): # Run had not fcupgated in HEL::scaler
            continue
        # ADDED ON SEPT 23RD TO CHECK UNGATED ASYMs
        #dfHELall["fcupgated"]=dfHELall["fcup"]
        #dfHELall["clockgated"]=dfHELall["clock"]
        
        #dfHELall["fcupgated"]=dfHELall["fcup"]
        #dfHELall["clockgated"]=dfHELall["clock"]
        r.append(run)
        print("Analyzing run",run)
        HWP.append(get_from_rcdb(run,'half_wave_plate'))
        Tpol.append(get_from_rcdb(run,'target_polarization'))
        if(HWP[-1]==0): # HWP in
            zhat = -1
        
        clockgated_pos.append(dfHELall.loc[dfHELall.helicity==zhat,"clockgated"].sum())
        fcupgated_pos.append(dfHELall.loc[dfHELall.helicity==zhat,"fcupgated"].sum())
        clockgated_neg.append(dfHELall.loc[dfHELall.helicity==-zhat,"clockgated"].sum())
        fcupgated_neg.append(dfHELall.loc[dfHELall.helicity==-zhat,"fcupgated"].sum())
            
        clockgated_pos_corr.append(dfHELall.loc[dfHELall.helicity==zhat,"raw_clockgated"].sum())
        fcupgated_pos_corr.append(dfHELall.loc[dfHELall.helicity==zhat,"raw_fcupgated"].sum())
        clockgated_neg_corr.append(dfHELall.loc[dfHELall.helicity==-zhat,"raw_clockgated"].sum())
        fcupgated_neg_corr.append(dfHELall.loc[dfHELall.helicity==-zhat,"raw_fcupgated"].sum())
        
        clockgated_asym_numerator.append(clockgated_pos[-1]-clockgated_neg[-1])
        clockgated_asym_denominator.append(clockgated_pos[-1]+clockgated_neg[-1])
        clockgated_asym.append(clockgated_asym_numerator[-1]/clockgated_asym_denominator[-1])
        fcupgated_asym_numerator.append(fcupgated_pos[-1]-fcupgated_neg[-1])
        fcupgated_asym_denominator.append(fcupgated_pos[-1]+fcupgated_neg[-1])
        fcupgated_asym.append(fcupgated_asym_numerator[-1]/fcupgated_asym_denominator[-1])
        
        clockgated_asym_numerator_corr.append(clockgated_pos_corr[-1]-clockgated_neg_corr[-1])
        clockgated_asym_denominator_corr.append(clockgated_pos_corr[-1]+clockgated_neg_corr[-1])
        clockgated_asym_corr.append(clockgated_asym_numerator_corr[-1]/clockgated_asym_denominator_corr[-1])
        fcupgated_asym_numerator_corr.append(fcupgated_pos_corr[-1]-fcupgated_neg_corr[-1])
        fcupgated_asym_denominator_corr.append(fcupgated_pos_corr[-1]+fcupgated_neg_corr[-1])
        fcupgated_asym_corr.append(fcupgated_asym_numerator_corr[-1]/fcupgated_asym_denominator_corr[-1])
        
        df.loc[len(df.index)] = [r[-1],HWP[-1],Tpol[-1],clockgated_asym[-1],fcupgated_asym[-1],clockgated_asym_corr[-1],fcupgated_asym_corr[-1]]
    
    return df
    
    
def asym_ana(df):
    
    r = df.run.to_numpy()
    HWP = df.HWP.to_numpy()
    Tpol = df.Tpol.to_numpy()
    clockgated_asym = df.clockgated_asym.to_numpy()
    fcupgated_asym = df.fcupgated_asym.to_numpy()
    clockgated_asym_corr = df.clockgated_asym_corr.to_numpy()
    fcupgated_asym_corr = df.fcupgated_asym_corr.to_numpy()
    
    fig1,axs1=plt.subplots(figsize=(20,10))
    axs1.plot(r,clockgated_asym,"k--")
    axs1.plot(r,clockgated_asym_corr,"r--")
    axs1.plot(r[HWP==0],clockgated_asym[HWP==0],"ko",label="No correction (HWP in)")
    axs1.plot(r[HWP==1],clockgated_asym[HWP==1],"ko",fillstyle='none',label="No correction (HWP out)")
    axs1.plot(r[HWP==0],clockgated_asym_corr[HWP==0],"ro",label="Correction (HWP in)")
    axs1.plot(r[HWP==1],clockgated_asym_corr[HWP==1],"ro",fillstyle='none',label="Correction (HWP out)")
    axs1.set_xlabel("Run Number",fontsize=20)
    axs1.set_ylabel(r"$\frac{CG^{+}-CG^{-}}{CG^{+}+CG^{-}}$",fontsize=20)
    axs1.set_title("Clockgated Asymmetry",fontsize=20)
    axs1.legend(fontsize=20)
    
    fig2,axs2=plt.subplots(figsize=(20,10))
    axs2.plot(r,fcupgated_asym,"k--")
    axs2.plot(r,fcupgated_asym_corr,"r--")
    axs2.plot(r[HWP==0],fcupgated_asym[HWP==0],"ko",label="No correction (HWP in)")
    axs2.plot(r[HWP==1],fcupgated_asym[HWP==1],"ko",fillstyle='none',label="No correction (HWP out)")
    axs2.plot(r[HWP==0],fcupgated_asym_corr[HWP==0],"ro",label="Correction (HWP in)")
    axs2.plot(r[HWP==1],fcupgated_asym_corr[HWP==1],"ro",fillstyle='none',label="Correction (HWP out)")
    axs2.set_xlabel("Run Number",fontsize=20)
    axs2.set_ylabel(r"$\frac{FC^{+}-FC^{-}}{FC^{+}+FC^{-}}$",fontsize=20)
    axs2.set_title("FCupgated Asymmetry",fontsize=20)
    axs2.legend(fontsize=20)
    
    fig3,axs3=plt.subplots(1,2,figsize=(20,10))
    axs3[0].plot(fcupgated_asym[(HWP==0) & (Tpol>0)],clockgated_asym[(HWP==0) & (Tpol>0)],"ko",markersize=10,label="HWP in , Tpol > 0")
    axs3[0].plot(fcupgated_asym[(HWP==0) & (Tpol<0)],clockgated_asym[(HWP==0) & (Tpol<0)],"ro",markersize=10,label="HWP in , Tpol < 0")
    axs3[0].plot(fcupgated_asym[(HWP==1) & (Tpol>0)],clockgated_asym[(HWP==1) & (Tpol>0)],"ko",fillstyle='none',markersize=10,label="HWP out , Tpol > 0")
    axs3[0].plot(fcupgated_asym[(HWP==1) & (Tpol<0)],clockgated_asym[(HWP==1) & (Tpol<0)],"ro",fillstyle='none',markersize=10,label="HWP out , Tpol < 0")
    axs3[0].set_xlabel("FCupgated Asymmetry",fontsize=20)
    axs3[0].set_ylabel("Clockgated Asymmetry",fontsize=20)
    axs3[0].set_title("Without Correction (Runs {:.0f}-{:.0f})".format(r[0],r[-1]),fontsize=20)
    max0 = np.amax(np.abs(axs3[0].get_xlim()+axs3[0].get_ylim()))
    axs3[0].set_xlim(-max0,max0)
    axs3[0].set_ylim(-max0,max0)
    axs3[0].legend(fontsize=15)
    axs3[0].grid()
    
    axs3[1].plot(fcupgated_asym_corr[(HWP==0) & (Tpol>0)],clockgated_asym_corr[(HWP==0) & (Tpol>0)],"ko",markersize=10,label="HWP in , Tpol > 0")
    axs3[1].plot(fcupgated_asym_corr[(HWP==0) & (Tpol<0)],clockgated_asym_corr[(HWP==0) & (Tpol<0)],"ro",markersize=10,label="HWP in , Tpol < 0")
    axs3[1].plot(fcupgated_asym_corr[(HWP==1) & (Tpol>0)],clockgated_asym_corr[(HWP==1) & (Tpol>0)],"ko",fillstyle='none',markersize=10,label="HWP out , Tpol > 0")
    axs3[1].plot(fcupgated_asym_corr[(HWP==1) & (Tpol<0)],clockgated_asym_corr[(HWP==1) & (Tpol<0)],"ro",fillstyle='none',markersize=10,label="HWP out , Tpol < 0")
    axs3[1].set_xlabel("FCupgated Asymmetry",fontsize=20)
    axs3[1].set_ylabel("Clockgated Asymmetry",fontsize=20)
    axs3[1].set_title("With Correction (Runs {:.0f}-{:.0f})".format(r[0],r[-1]),fontsize=20)
    max1 = np.amax(np.abs(axs3[1].get_xlim()+axs3[1].get_ylim()))
    axs3[1].set_xlim(-max1,max1)
    axs3[1].set_ylim(-max1,max1)
    axs3[1].legend(fontsize=15)
    axs3[1].grid()

In [1]:
def linear_fit(x,a,b):
    return a*x+b

'''def scaler_ana_manyRuns(filepath = "",
                        runMin = 0,
                        runMax = 0):
    
    # Default filepath
    if(filepath==""):
        filepath = "/volatile/clas12/users/gmat/clas12analysis.sidis.data/rgc-scaler/"

    
    dfRUN = pd.DataFrame()
    dfHEL = pd.DataFrame()
    
    for run in np.arange(runMin,runMax+1):
        # Get CSV scalers
        csvRUN, csvHEL = get_csv(filepath,run)
        if(not exists(csvRUN)):
            continue

        # Create dataframes
        dfRUNtmp = pd.read_csv(csvRUN,header=None,
                            names=["run","fileidx","entries","fcupgated_min","fcupgated_max",
                                  "fcg_diff","livetime_total","livetime_avg","fcup_min","fcup_max","fc_diff","livetime_avg*fc_diff"])

        dfHELtmp = pd.read_csv(csvHEL,header=None,
                            names=["run","fileidx","entries","fcupgated_total","fcupgated_total_pos",
                                  "fcupgated_total_neg","fcupgated_total_zero"])

        # Grab RCDB values of interest
        HWP = get_from_rcdb(run,'half_wave_plate')
        if(HWP==0): # If the HWP is in, swap HEL::scaler columns
            dfHELtmp["temp"]=dfHELtmp["fcupgated_total_pos"]
            dfHELtmp["fcupgated_total_pos"]=dfHELtmp["fcupgated_total_neg"]
            dfHELtmp["fcupgated_total_neg"]=dfHELtmp["temp"]

        # Append to main DataFrame
        dfRUN = dfRUN.append(dfRUNtmp)
        dfHEL = dfHEL.append(dfHELtmp)
    
    
    
    # ----------------------------
    # PLOTS
    # ----------------------------
    # Asymmetry vs. Faulty file %-age
    fig1,axs1 = plt.subplots(1,1,figsize=(10,10))
    runs=[]
    faulty=[]
    asym = []
    for r in dfHEL["run"].unique():
        df = dfHEL[dfHEL.run==r]
        runs.append(r)
        faulty.append(len(df[np.abs(df.fcupgated_total)<3]) / len(df))
        asym.append(np.abs(np.sum(df.fcupgated_total_pos-df.fcupgated_total_neg)/np.sum(df.fcupgated_total_pos+df.fcupgated_total_neg)))
    
    axs1.plot(faulty,asym,"k+")
    axs1.set_xlabel("Percentage faulty FCup files")
    axs1.set_ylabel("|fcupgated asymmetry|")

    #pd.set_option("max_rows",None)
    #display(pd.DataFrame({"run":runs,"faulty%age":faulty,"asym":asym}))
    
    
    # Mean FCup total (pedestal) vs. Mean FCup total (valley)
    fig2,axs2 = plt.subplots(1,1,figsize=(10,10))
    runs=[]
    pedestal=[]
    valley=[]
    pedestalerr=[]
    valleyerr=[]
    for r in dfHEL["run"].unique():
        df = dfHEL[dfHEL.run==r]
        dfp = df[np.abs(df.fcupgated_total>3)]
        dfv = df[np.abs(df.fcupgated_total<3)]
        runs.append(r)
        # For some reason, run 16947 is safe
        if(r==16947):
            continue
        pedestal.append(np.mean(dfp.fcupgated_total)) 
        valley.append(np.mean(dfv.fcupgated_total)) 
        pedestalerr.append(np.std(dfp.fcupgated_total)/np.sqrt(len(dfp)))
        valleyerr.append(np.std(dfv.fcupgated_total)/np.sqrt(len(dfv)))
    axs2.errorbar(pedestal,valley,xerr=pedestalerr,yerr=valleyerr,fmt="k+")
    axs2.set_xlabel("<FCupgated total> (pedestal)")
    axs2.set_ylabel("<FCupgated total> (valley)")

    params,covar = curve_fit(linear_fit,pedestal,valley,sigma=valleyerr)
    errors = np.sqrt(np.diag(covar))
    xvals = np.arange(np.amin(pedestal),np.amax(pedestal)+1)
    axs2.plot(xvals,linear_fit(xvals,params[0],params[1]),"r-",label="y=mx+b\nm={:.4f}+/-{:.4f}\nb={:.3f}+/-{:.3f}".format(params[0],errors[0],params[1],errors[1]))
    axs2.legend()
    
    
    # Mean FCup pos (pedestal) vs. Mean FCup pos (valley)
    fig3,axs3 = plt.subplots(1,1,figsize=(10,10))
    runs=[]
    pedestal=[]
    valley=[]
    pedestalerr=[]
    valleyerr=[]
    for r in dfHEL["run"].unique():
        df = dfHEL[dfHEL.run==r]
        dfp = df[np.abs(df.fcupgated_total_pos>1.5)]
        dfv = df[np.abs(df.fcupgated_total_pos<1.5)]
        runs.append(r)
        # For some reason, run 16947 is safe
        if(r==16947):
            continue
        pedestal.append(np.mean(dfp.fcupgated_total_pos)) 
        valley.append(np.mean(dfv.fcupgated_total_pos)) 
        pedestalerr.append(np.std(dfp.fcupgated_total_pos)/np.sqrt(len(dfp)))
        valleyerr.append(np.std(dfv.fcupgated_total_pos)/np.sqrt(len(dfv)))
    axs3.errorbar(pedestal,valley,xerr=pedestalerr,yerr=valleyerr,fmt="k+")
    axs3.set_xlabel("<FCupgated total (h=+1)> (pedestal)")
    axs3.set_ylabel("<FCupgated total (h=+1)> (valley)")

    params,covar = curve_fit(linear_fit,pedestal,valley,sigma=valleyerr)
    errors = np.sqrt(np.diag(covar))
    xvals = np.arange(np.amin(pedestal),np.amax(pedestal)+1)
    axs3.plot(xvals,linear_fit(xvals,params[0],params[1]),"r-",label="y=mx+b\nm={:.4f}+/-{:.4f}\nb={:.3f}+/-{:.3f}".format(params[0],errors[0],params[1],errors[1]))
    axs3.legend()
    
    
    # Mean FCup neg (pedestal) vs. Mean FCup  neg (valley)
    fig4,axs4 = plt.subplots(1,1,figsize=(10,10))
    runs=[]
    pedestal=[]
    valley=[]
    pedestalerr=[]
    valleyerr=[]
    for r in dfHEL["run"].unique():
        df = dfHEL[dfHEL.run==r]
        dfp = df[np.abs(df.fcupgated_total_neg>1.5)]
        dfv = df[np.abs(df.fcupgated_total_neg<1.5)]
        runs.append(r)
        # For some reason, run 16947 is safe
        if(r==16947):
            continue
        pedestal.append(np.mean(dfp.fcupgated_total_neg)) 
        valley.append(np.mean(dfv.fcupgated_total_neg)) 
        pedestalerr.append(np.std(dfp.fcupgated_total_neg)/np.sqrt(len(dfp)))
        valleyerr.append(np.std(dfv.fcupgated_total_neg)/np.sqrt(len(dfv)))
    axs4.errorbar(pedestal,valley,xerr=pedestalerr,yerr=valleyerr,fmt="k+")
    axs4.set_xlabel("<FCupgated total (h=-1)> (pedestal)")
    axs4.set_ylabel("<FCupgated total (h=-1)> (valley)")

    params,covar = curve_fit(linear_fit,pedestal,valley,sigma=valleyerr)
    errors = np.sqrt(np.diag(covar))
    xvals = np.arange(np.amin(pedestal),np.amax(pedestal)+1)
    axs4.plot(xvals,linear_fit(xvals,params[0],params[1]),"r-",label="y=mx+b\nm={:.4f}+/-{:.4f}\nb={:.3f}+/-{:.3f}".format(params[0],errors[0],params[1],errors[1]))
    axs4.legend()
    
    
'''

In [ ]:
def get_fc_vals(filepath = "",
                runMin = 0,
                runMax = 0):
    # Default filepath
    if(filepath==""):
        filepath = "/volatile/clas12/users/gmat/clas12analysis.sidis.data/rgc-scaler/"

    
    dfRUN = pd.DataFrame()
    dfHEL = pd.DataFrame()
    
    for run in np.arange(runMin,runMax+1):
        # Get CSV scalers
        csvRUN, csvHEL = get_csv(filepath,run)
        if(not exists(csvRUN)):
            continue

        # Create dataframes
        dfRUNtmp = pd.read_csv(csvRUN,header=None,
                            names=["run","fileidx","entries","fcupgated_min","fcupgated_max",
                                  "fcg_diff","livetime_total","livetime_avg","fcup_min","fcup_max","fc_diff","livetime_avg*fc_diff"])

        dfHELtmp = pd.read_csv(csvHEL,header=None,
                            names=["run","fileidx","entries","fcupgated_total","fcupgated_total_pos",
                                  "fcupgated_total_neg","fcupgated_total_zero"])

        # Grab RCDB values of interest
        HWP = get_from_rcdb(run,'half_wave_plate')
        if(HWP==0): # If the HWP is in, swap HEL::scaler columns
            dfHELtmp["temp"]=dfHELtmp["fcupgated_total_pos"]
            dfHELtmp["fcupgated_total_pos"]=dfHELtmp["fcupgated_total_neg"]
            dfHELtmp["fcupgated_total_neg"]=dfHELtmp["temp"]
        
        # scale properly
        where=dfHELtmp["fcupgated_total"]<3
        dfHELtmp.loc[where,"fcupgated_total"] *= 75.758
        dfHELtmp.loc[where,"fcupgated_total_pos"] *= 75.758
        dfHELtmp.loc[where,"fcupgated_total_neg"] *= 75.758
        dfHELtmp.loc[where,"fcupgated_total_zero"] *= 75.758
        # Append to main DataFrame
        dfRUN = dfRUN.append(dfRUNtmp)
        dfHEL = dfHEL.append(dfHELtmp)
    
    dfRUN.to_csv("RUN_test.csv")
    dfHEL.to_csv("HEL_test.csv")